In [103]:
from strands import Agent, tool
from strands_tools import use_llm, memory
import boto3
import json_repair
import json
import sagemaker
from strands.models import BedrockModel


boto_session = boto3.session.Session()
sess = sagemaker.Session(boto_session=boto_session)

# Initialize bedrock agent runtime
bedrock_agent_runtime = boto_session.client("bedrock-agent-runtime")

# Define knowledge base parameters
knowledge_base_name = "compliance-kb"
knowledge_base_description = "knowledge base contain guideline requirements and output schema for different type of video posts"
new_bucket = f"{knowledge_base_name}-{sess.boto_region_name}"
# Bedrock
bedrock_model = BedrockModel(
  model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
  temperature=0.1,
)


In [10]:
import sys
sys.path.insert(0, '..')

from helper.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)

kb = KnowledgeBasesForAmazonBedrock()

Build a knowledge base

In [11]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    new_bucket
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

Knowledge Base compliance-kb already exists.
Retrieved Knowledge Base Id: L933KIHBDY
Retrieved Data Source Id: UCXUDJMRMW
Knowledge Base ID: L933KIHBDY
Data Source ID: UCXUDJMRMW
CPU times: user 20.8 ms, sys: 4.18 ms, total: 24.9 ms
Wall time: 609 ms


In [12]:
standards_folder = "standards"

!aws s3 sync {standards_folder} s3://{new_bucket}/

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

{ 'dataSourceId': 'UCXUDJMRMW',
  'ingestionJobId': 'VE4QBEXZE4',
  'knowledgeBaseId': 'L933KIHBDY',
  'startedAt': datetime.datetime(2025, 5, 18, 13, 45, 45, 923221, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 5, 18, 13, 45, 45, 923221, tzinfo=tzutc())}
{ 'dataSourceId': 'UCXUDJMRMW',
  'ingestionJobId': 'VE4QBEXZE4',
  'knowledgeBaseId': 'L933KIHBDY',
  'startedAt': datetime.datetime(2025, 5, 18, 13, 45, 45, 923221, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 3,
        

### Compliance Analyzer

In [16]:
@tool
def compliance_analyzer(query: str) -> str:
    """Analyze compliance standards, and extract relevant insights from the docs"""
    print("📄 Compliance anlayizer analyzes documentation...")

    @tool
    def retreive_docs(query: str, max_results=1) -> str:
        try:
            # Call the retrieve API
            response = bedrock_agent_runtime.retrieve(
                knowledgeBaseId=kb_id,
                retrievalQuery={
                    'text': query
                },
                retrievalConfiguration={
                    'vectorSearchConfiguration': {
                        'numberOfResults': max_results
                    }
                }
            )
            
            # Extract and return the retrieved results
            return json.dumps(response.get('retrievalResults', []))
        
        except Exception as e:
            print(f"Error retrieving from knowledge base: {e}")
            return ""

        
    # System prompt for generating answers from retrieved information
    ANSWER_SYSTEM_PROMPT = """
    You are a compliance documentation analyzer that identifies and analyzes relevant compliance document base on 
    the task. Upon receiving a task, you will locate the most appropriate compliance documents, summarize 
    the key compliance requirements, and extract the corresponding JSON structure as specified in the documentation. 
    You output will be JSON ONLY. The JSON only has a "requriements" attribute is a list of mandatory requirements and a "schema" 
    filed, which is the complete JSON example.
    """

    agent = Agent(model=bedrock_model,
                  tools=[retreive_docs],
                  system_prompt=ANSWER_SYSTEM_PROMPT)

    return  agent(query)

result = compliance_analyzer("preparing this filem for social media post")

📈 Compliance anlayizer analyzes documentation...
Let me search for relevant compliance documentation regarding social media file preparation.
Tool #9: retreive_docs


Based on the retrieved documentation, I'll provide the compliance requirements in JSON format:

{
    "requirements": [
        "Description must be less than 75 words",
        "Description must follow Hook + Main Point + Call-to-action format",
        "Must include film name, cast, and roles (for Films) or venue name, teams, and athletes (for Sports)",
        "Must use casual tone",
        "Must include 2-3 hashtags",
        "Must include relevant @mentions",
        "Emoji usage when applicable"
    ],
    "schema": {
        "description": "[Insert Social Media Copy]"
    }
}

### Synopsis Agent

In [ ]:
@tool
def synopsis_agent(query: str) -> str:
    """generate and enhace a factually correct and compliant video metadata based on the requirements, feedback, and expected output format."""
    print("✨ Synopsis agent creating ...")

    @tool
    def create_metadata(video_analysis: str, additional_info: str, requirements: str, output: str) -> str:
        
        AGENT_SYSTEM_PROMPT = """
        You are a media operations specialist responsible for analyzing raw analysis of a video and additional
        information about the video, the produce a compliant metadata following the requirements and output format.
        """
        agent = Agent(model=bedrock_model, system_prompt=AGENT_SYSTEM_PROMPT)

        prompt = """
            Here is the video analysis: 
            {video_analysis}
            
            Here is additional info about the video: 
            {additional_info}

            Here is the requirements:
            {requirements}

            Please generate and output the metadata in following output format ONLY.
            {output}
        """.format(video_analysis=video_analysis, additional_info=additional_info, requirements=requirements, output=output)

        return agent(prompt=prompt)
    
    @tool
    def enhance_metadata(current_metadata: str, feedback: str, video_analysis:str, additional_info: str, requirements: str, output: str) -> str:
        AGENT_SYSTEM_PROMPT = """
         You are a media operations specialist responsible for enhancing existing video metadata base on feedback. your are provided
         with the original video analysis for reference, as well as the requirements and output format to follow. Finally output needs
         to account for all the provided information.
         """
         
        agent = Agent(model=bedrock_model, system_prompt=AGENT_SYSTEM_PROMPT)

        prompt = """
            Here is the current metadata: {current_metadata}
            Here is the feedback: {feedback}
            Here is the video analysis: {video_analysis}
            Here is additional info about the video: {additional_info}
            Here is the requirements: {requirements}
            Please generate and output the metadata in following output format ONLY.
            {output}
        """.format(current_metadata=current_metadata, feedback=feedback, video_analysis=video_analysis, requirements=requirements, output=output)

        return agent(prompt=prompt)
    
    # System prompt for generating answers from retrieved information
    ANSWER_SYSTEM_PROMPT = """
    You are a media operations specialist, Your task is to generate or enhace a factually correct and compliant video metadata based on provided information.
    
    Core behavior:
    1. You should always create the metadata based on available information, do not manufacture facts on your own. 
    2. You should always provide clear, direct answers and present information in a easy-to-understand manner.
    3. Do not ask user for additioanl detail if you can not find it in the available information.
    4. Output metadata ONLY, skip explanation.
    """

    agent = Agent(model=bedrock_model,
                  tools=[create_metadata, enhance_metadata],
                  system_prompt=ANSWER_SYSTEM_PROMPT)

    return agent(query)

In [64]:
test_input = """
Here is the raw video analysis:
{'visual_summary': 'In a series of mysterious disappearances near El Matador beach, Captain Foster of the LAPD investigates the vanishing of Detective Sullivan and several other men. The cases share an eerie connection to a distinctive rock formation where the ocean meets the shore, described as resembling a mythic sea creature.\n\nThe investigation reveals that unlike previous incidents in the area, which were associated with mob activity under Mickey Cohen, these disappearances involve seemingly ordinary men - a school teacher, an insurance salesman, and a retiree. The only common thread among them is that they were all divorced, with minimal criminal history aside from one old DUI case.\n\nWhile some suggest suicide as an explanation, the absence of bodies and a credible witness account points to something more mysterious. The witness, described as a trustworthy "Eagle Scout," reported a disturbing incident while walking his dog along the bluffs above El Matador. He observed a man peculiarly searching the large rock formation, apparently looking for something or someone. Suddenly, unprecedented weather conditions emerged - darkness fell rapidly, accompanied by thunder and lightning.\n\nWhen the witness looked back at the rock, the man had vanished. In his place stood a woman in a white dress with alabaster skin, described as beautiful but otherworldly, as if from another time and place. She was seen staring directly at the witness before he fled from the intensifying storm. This testimony, though initially dismissed as outlandish, has become a crucial element in understanding these unexplained disappearances.',
 'audio_transcription': "   [spk_0]: So these guys just disappeared. [spk_1]: Yeah, on that stretch of road right above El Matador. You know it. With the big rock, that's right,\n\n[spk_2]: yeah. You know, Mickey Cohen used to take his associates out there, get him a bond voyage.\n\n[spk_1]: Mob heads? No, not this time. One of these guys was a school teacher. Next guy, he sold insurance, and the last guy, he was retired. Any connection? Mm. I mean, I checked them all out. There's no criminal records, no association in any criminal organizations. One of the guys was arrested for drunk driving a long time ago. They were all divorced.\n\n[spk_2]: Well, in that case, maybe you don't want to go out there, boss.\n\n[spk_1]: What are you, a comedian? If that's the case, you sure don't want to go out there.\n\n[spk_2]: Yeah, look, I know what happened, all right? They go out to the rock, they take that long, cold, slow swim to\n\n[spk_1]: China. Suicide.\n\n[spk_2]: It's been known to happen out there.\n\n[spk_1]: Uh, not this time. What do you mean? Well, for one thing, there's no bodies, and Somebody was out there the day the last guy disappeared.\n\n[spk_2]: A witness,\n\n[spk_1]: yeah, a witness.\n\n[spk_2]: Why didn't you say that in the first place?\n\n[spk_1]: Because I thought the guy was nuts, what he said.\n\n[spk_2]: But now you believe him.\n\n[spk_1]: I had him checked out too. The guy's a goddamn Eagle Scout. So now, now I don't know what to believe. I know I don't want to believe him.\n\n[spk_2]: What did he say?\n\n[spk_1]: He said he was out there walking his dog along the bluffs above El Matador, right? And he looks down at the beach at that huge rock, you know, the one that's separated from the mainland right where the ocean meets the shore. His words looks like a, like a mythic sea creature, you know that one. Anyway, he looks down there and there's a guy who's on top of that rock, and he's thinking to himself, how the hell do you get up there? You gotta be a goddamn mountain climber. But more importantly, he says the guy's acting really funny. He's looking all over that rock like he's looking for something or someone. At that very moment, he says it got really, really dark, really fast, and bombs start going off in the air, thunder and lightning, weather like he'd never seen before coming from he didn't know where. When he looks back at the rock, the guy's gone. But there's a woman there. She's in a white dress, alabaster skin, his words, and he says she's staring right up at him. He sees her, she's beautiful. But like she's from some other time, some other place. He looks back at the weather cause it's getting worse and worse. He decides to hightail it out of there. And that was it. That was his story, right? [spk_1]: So now Now I need you to go up there and find that woman. [spk_0]: So these guys just disappear.\n\n[spk_1]: No, not this time. Somebody was out there the day the last guy disappeared. A witness, yeah, a witness. He said that he was up there walking his dog along those bluffs above El Matador, and there's a guy who's on top of that rock. He's just like looking all over the rock as if he's looking for someone or something. Then it gets dark all of a sudden, really fast. Like thunder and lightning from everywhere, all this weather. When he looks back at the rock, the guy's gone, but there's a woman there in a long white dress with alabaster skin. And she's staring right up at him, not moving. Now I don't know what to [spk_1]: I know I sure don't want to believe him.   [spk_3]: Yeah, dispatch. This is Captain Foster. I'm out here at the El Matador location. There's no sign of Detective Sullivan. Now what's this car. Listen, I'm gonna take a look around before you send anybody out here, so. Hang on a minute.\n\n[spk_4]: Captain Foster, [spk_4]: you come in please. Captain Foster I. LAPD, put your hands in the air. Put your hands up now. Hm    ",
 'metadata': {'asset_id': '0',
  'semantic_modality': 'VIDEO',
  's3_bucket': 'sagemaker-us-west-2-376678947624',
  's3_key': 'media-operations-agent-claude/NetflixMeridian.mp4',
  'format': 'QuickTime / MOV',
  'frame_rate': 29.97,
  'codec': 'h264',
  'duration_millis': 718918,
  'frame_width': 1280,
  'frame_height': 720}}

Here is additional information found:
This clip is from the film "Meridian" (2016). The clip features Kevin Kilner as Captain Foster investigating mysterious disappearances near a beach, with Reid Scott as Detective Sullivan and Elyse Levesque as the mysterious woman.

Here is the requirements: 
  "Description must be less than 75 words",
  "Description must follow Hook + Main Point + Call-to-action format",
  "Must include film name, cast, and roles (for Films) or venue name, teams, and athletes (for Sports)",
  "Must use casual tone",
  "Must include 2-3 hashtags",
  "Must include relevant @mentions",
  "Emoji usage when applicable"

Here is the output format:
  {
      "description": "[Insert Social Media Copy]"
  }
"""

synopsis_agent(test_input)

📈 Synopsis agent creating ...

Tool #143: create_metadata

Tool #145: create_metadata

Tool #147: create_metadata

Tool #149: create_metadata
{
    "description": "👻 Something spooky is happening at El Matador beach! Watch @KevinKilner as Captain Foster investigate mysterious disappearances in 'Meridian' (2016), featuring @ReidScott as Detective Sullivan and @ELevesque as the enigmatic woman in white. Don't miss this supernatural thriller! #Meridian #Supernatural"
}

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': []}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'create_metadata': ToolMetrics(tool={'toolUseId': 'tooluse_Fn1tb9LXSS-4bOMWMZJQWQ', 'name': 'create_metadata', 'input': {'video_analysis': '{\'visual_summary\': \'In a series of mysterious disappearances near El Matador beach, Captain Foster of the LAPD investigates the vanishing of Detective Sullivan and several other men. The cases share an eerie connection to a distinctive rock formation where the ocean meets the shore, described as resembling a mythic sea creature.\n\nThe investigation reveals that unlike previous incidents in the area, which were associated with mob activity under Mickey Cohen, these disappearances involve seemingly ordinary men - a school teacher, an insurance salesman, and a retiree. The only common thread among them is that they were all divorced, with minimal criminal history aside from one old DUI case.\n\nWhile some suggest 

### QC Agent

In [71]:
@tool
def qc_agent(query: str) -> str:
    """QC check video metadata against the requirements and expected output schema."""
    print("✅ QC agent reviewing...")

    @tool
    def word_count(text: str) -> int:
        """Count words in text.

        This docstring is used by the LLM to understand the tool's purpose.
        """      
        return len(text.split())
    
    # System prompt for generating answers from retrieved information
    ANSWER_SYSTEM_PROMPT = """
    You are a Quality Check (QC) Agent, You task is to conduct a thorough review of the video
    metadata against requirements and expected output format. Report hard requirement violations as feedback.
    
    Core behavior:
    1. You should always review base on available information, do not manufacture facts on your own. 
    2. You should always provide clear, direct answers and present information in a easy-to-understand manner.
    3. Do not ask user for additioanl detail if you can not find it in the available information.
    4. Only report hard requirement violations as feedback. If None, just say "Pass". Skip any explanation.
    """

    agent = Agent(model=bedrock_model,
                  tools=[word_count],
                  system_prompt=ANSWER_SYSTEM_PROMPT)

    return agent(query)

In [72]:
test_input = """
Here is the metadata:
{
    "description": "👻 Something spooky is happening at El Matador beach! Watch @KevinKilner as Captain Foster investigate mysterious disappearances in 'Meridian' (2016), featuring @ReidScott as Detective Sullivan and @ELevesque as the enigmatic woman in white. Don't miss this supernatural thriller! #Meridian #Supernatural"
}

Here is the requirements:
  "Description must be less than 75 words",
  "Description must follow Hook + Main Point + Call-to-action format",
  "Must include film name, cast, and roles (for Films) or venue name, teams, and athletes (for Sports)",
  "Must use casual tone",
  "Must include 2-3 hashtags",
  "Must include relevant @mentions",
  "Emoji usage when applicable" 

Here is the output format:
  {
      "description": "[Insert Social Media Copy]"
  }
"""

qc_agent(test_input)

✅ QC agent reviewing...
Let me check the word count first and then review against all requirements.
Tool #232: word_count


Pass

The metadata meets all requirements:
1. Word count is 40 (under 75)
2. Has Hook (spooky happening), Main Point (movie plot), and CTA (Don't miss)
3. Includes film name (Meridian), cast (@KevinKilner, @ReidScott, @ELevesque) and roles (Captain Foster, Detective Sullivan, woman in white)
4. Uses casual tone
5. Has 2 hashtags (#Meridian #Supernatural)
6. Includes 3 @mentions
7. Uses emoji (👻)
8. Follows correct output format

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "\n\nPass\n\nThe metadata meets all requirements:\n1. Word count is 40 (under 75)\n2. Has Hook (spooky happening), Main Point (movie plot), and CTA (Don't miss)\n3. Includes film name (Meridian), cast (@KevinKilner, @ReidScott, @ELevesque) and roles (Captain Foster, Detective Sullivan, woman in white)\n4. Uses casual tone\n5. Has 2 hashtags (#Meridian #Supernatural)\n6. Includes 3 @mentions\n7. Uses emoji (👻)\n8. Follows correct output format"}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'word_count': ToolMetrics(tool={'toolUseId': 'tooluse_qPJz0uTgTh6xcZpc_8H0rQ', 'name': 'word_count', 'input': {'text': "👻 Something spooky is happening at El Matador beach! Watch @KevinKilner as Captain Foster investigate mysterious disappearances in 'Meridian' (2016), featuring @ReidScott as Detective Sullivan and @ELevesque as the enigmatic woman in white. Don't miss this supernatural thriller! #Meridi

### Supervisor Agent

In [76]:
# Level 1 - Executive Coordinator
COORDINATOR_SYSTEM_PROMPT = """
You are an executive coordinator who oversees complex analyses across multiple domains.
For compliance questions, use compliance_analyzer tool.
For metadata generataion and enhancement, use synopsis_agent tool.
For quality check, use qc_agent tool.

You process should be:
1. analyze the raw video analysis and additional info to determine what type of video (e.g.: Film or Sports)
2. get the compliance requirements and output format base on the user define task
3. generate or enhacement video metadata base on the requirements and ouptut format.
4. repeat step 3 until you pass the quality check.
5. return the final metadata in the compliant JSON output format. Skip any explanation
"""

In [77]:
# Create the coordinator agent with all tools
supervisor = Agent(
    model=bedrock_model,
    system_prompt=COORDINATOR_SYSTEM_PROMPT,
    tools=[compliance_analyzer, synopsis_agent, qc_agent],
    callback_handler=None
)


In [110]:
task = "company websit post" # "social media post"
test_input="""
The task is to create a compliant {TASK} for the following video

Here is the raw video analysis:
{'visual_summary': 'In a series of mysterious disappearances near El Matador beach, Captain Foster of the LAPD investigates the vanishing of Detective Sullivan and several other men. The cases share an eerie connection to a distinctive rock formation where the ocean meets the shore, described as resembling a mythic sea creature.\n\nThe investigation reveals that unlike previous incidents in the area, which were associated with mob activity under Mickey Cohen, these disappearances involve seemingly ordinary men - a school teacher, an insurance salesman, and a retiree. The only common thread among them is that they were all divorced, with minimal criminal history aside from one old DUI case.\n\nWhile some suggest suicide as an explanation, the absence of bodies and a credible witness account points to something more mysterious. The witness, described as a trustworthy "Eagle Scout," reported a disturbing incident while walking his dog along the bluffs above El Matador. He observed a man peculiarly searching the large rock formation, apparently looking for something or someone. Suddenly, unprecedented weather conditions emerged - darkness fell rapidly, accompanied by thunder and lightning.\n\nWhen the witness looked back at the rock, the man had vanished. In his place stood a woman in a white dress with alabaster skin, described as beautiful but otherworldly, as if from another time and place. She was seen staring directly at the witness before he fled from the intensifying storm. This testimony, though initially dismissed as outlandish, has become a crucial element in understanding these unexplained disappearances.',
 'audio_transcription': "   [spk_0]: So these guys just disappeared. [spk_1]: Yeah, on that stretch of road right above El Matador. You know it. With the big rock, that's right,\n\n[spk_2]: yeah. You know, Mickey Cohen used to take his associates out there, get him a bond voyage.\n\n[spk_1]: Mob heads? No, not this time. One of these guys was a school teacher. Next guy, he sold insurance, and the last guy, he was retired. Any connection? Mm. I mean, I checked them all out. There's no criminal records, no association in any criminal organizations. One of the guys was arrested for drunk driving a long time ago. They were all divorced.\n\n[spk_2]: Well, in that case, maybe you don't want to go out there, boss.\n\n[spk_1]: What are you, a comedian? If that's the case, you sure don't want to go out there.\n\n[spk_2]: Yeah, look, I know what happened, all right? They go out to the rock, they take that long, cold, slow swim to\n\n[spk_1]: China. Suicide.\n\n[spk_2]: It's been known to happen out there.\n\n[spk_1]: Uh, not this time. What do you mean? Well, for one thing, there's no bodies, and Somebody was out there the day the last guy disappeared.\n\n[spk_2]: A witness,\n\n[spk_1]: yeah, a witness.\n\n[spk_2]: Why didn't you say that in the first place?\n\n[spk_1]: Because I thought the guy was nuts, what he said.\n\n[spk_2]: But now you believe him.\n\n[spk_1]: I had him checked out too. The guy's a goddamn Eagle Scout. So now, now I don't know what to believe. I know I don't want to believe him.\n\n[spk_2]: What did he say?\n\n[spk_1]: He said he was out there walking his dog along the bluffs above El Matador, right? And he looks down at the beach at that huge rock, you know, the one that's separated from the mainland right where the ocean meets the shore. His words looks like a, like a mythic sea creature, you know that one. Anyway, he looks down there and there's a guy who's on top of that rock, and he's thinking to himself, how the hell do you get up there? You gotta be a goddamn mountain climber. But more importantly, he says the guy's acting really funny. He's looking all over that rock like he's looking for something or someone. At that very moment, he says it got really, really dark, really fast, and bombs start going off in the air, thunder and lightning, weather like he'd never seen before coming from he didn't know where. When he looks back at the rock, the guy's gone. But there's a woman there. She's in a white dress, alabaster skin, his words, and he says she's staring right up at him. He sees her, she's beautiful. But like she's from some other time, some other place. He looks back at the weather cause it's getting worse and worse. He decides to hightail it out of there. And that was it. That was his story, right? [spk_1]: So now Now I need you to go up there and find that woman. [spk_0]: So these guys just disappear.\n\n[spk_1]: No, not this time. Somebody was out there the day the last guy disappeared. A witness, yeah, a witness. He said that he was up there walking his dog along those bluffs above El Matador, and there's a guy who's on top of that rock. He's just like looking all over the rock as if he's looking for someone or something. Then it gets dark all of a sudden, really fast. Like thunder and lightning from everywhere, all this weather. When he looks back at the rock, the guy's gone, but there's a woman there in a long white dress with alabaster skin. And she's staring right up at him, not moving. Now I don't know what to [spk_1]: I know I sure don't want to believe him.   [spk_3]: Yeah, dispatch. This is Captain Foster. I'm out here at the El Matador location. There's no sign of Detective Sullivan. Now what's this car. Listen, I'm gonna take a look around before you send anybody out here, so. Hang on a minute.\n\n[spk_4]: Captain Foster, [spk_4]: you come in please. Captain Foster I. LAPD, put your hands in the air. Put your hands up now. Hm    ",
 'metadata': {'asset_id': '0',
  'semantic_modality': 'VIDEO',
  's3_bucket': 'sagemaker-us-west-2-376678947624',
  's3_key': 'media-operations-agent-claude/NetflixMeridian.mp4',
  'format': 'QuickTime / MOV',
  'frame_rate': 29.97,
  'codec': 'h264',
  'duration_millis': 718918,
  'frame_width': 1280,
  'frame_height': 720}}

Here is additional information found:
This clip is from the film "Meridian" (2016). The clip features Kevin Kilner as Captain Foster investigating mysterious disappearances near a beach, with Reid Scott as Detective Sullivan and Elyse Levesque as the mysterious woman.
""".replace("{TASK}", task)
output = supervisor(test_input)

json_output = json_repair.loads(output.message['content'][0]['text'])
json_output

📈 Synopsis agent creating ...

Tool #307: create_metadata
Meridian

When a routine beach patrol turns into a supernatural enigma, Captain Foster (Kevin Kilner) finds himself drawn into a mysterious case at the scenic El Matador Beach. After discovering inexplicable phenomena along the shoreline, he partners with the determined Detective Sullivan (Reid Scott) to unravel a series of otherworldly events that defy explanation.

Their investigation takes an unexpected turn when they encounter a enigmatic woman (Elyse Levesque) whose presence seems intrinsically linked to the beach's growing mysteries. As the boundaries between natural and supernatural blur, they must race against time to uncover the truth behind the forces at work on these haunting shores.

Ready to dive into a supernatural mystery that will challenge everything you believe? Watch Meridian now and discover what lies beneath the surface of El Matador Beach.✅ QC agent reviewing...
Let me check the word count requirements for 

{'title': "Mysterious Disappearances Haunt El Matador Beach in 'Meridian'",
 'description': "When a series of inexplicable disappearances plague the iconic El Matador Beach, LAPD Captain Foster (Kevin Kilner) finds himself drawn into a supernatural mystery that defies explanation. After his partner Detective Sullivan (Reid Scott) vanishes while investigating the case, Foster discovers an unsettling pattern connecting the missing men to a mysterious rock formation where the ocean meets the shore.\n\n    As the investigation deepens, a credible witness reports encountering an otherworldly woman (Elyse Levesque) in a white dress, whose ethereal presence may hold the key to the disappearances. With each new revelation, Captain Foster must confront the possibility that something beyond the natural world is at work on this stretch of California coastline.\n\n    Experience this gripping supernatural thriller that blends police procedural with otherworldly mystery. Stream 'Meridian' now and u

### Clean Up

In [111]:
kb.delete_kb(knowledge_base_name)

Data Source deleted successfully!
Knowledge Base deleted successfully!
'NoneType' object has no attribute 'indices'
OpenSource Collection Index deleted successfully!
OpenSource Serveless access policy deleted successfully!
OpenSource Serveless network policy deleted successfully!
OpenSource Serveless encryption policy deleted successfully!
Knowledge Base S3 bucket deleted successfully!
Knowledge Base Roles and Policies deleted successfully!
Resources deleted successfully!
